In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=bd1baa2240cb05a6891d33a2f33ed1b65c432b1324b3ca080d054fc1b580e613
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Downloading and preprocessing the data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Product_Feature_ner/data_bio_shuffled.csv", encoding='unicode_escape')
data.head()

,sentence_id,words,labels
0,0,,O
1,0,Fly,O
2,0,Me,O
3,0,to,O
4,0,Leanin,B-BRAND


In [ ]:
#Let's check how many sentences and words (and corresponding tags) there are in this dataset:
data.count()

sentence_id    1139040
words          1139023
labels         1139040
dtype: int64

In [ ]:
#Let's have a look at the different NER tags, and their frequency:
print("Number of tags: {}".format(len(data.labels.unique())))
frequencies = data.labels.value_counts()
frequencies

Number of tags: 4


O          988665
B-BRAND     97677
I-BRAND     51157
-            1541
Name: labels, dtype: int64

In [ ]:
#Remove special characters
entities_to_remove = ["-"]
data = data[~data.labels.isin(entities_to_remove)]
data.head()

,sentence_id,words,labels
0,0,,O
1,0,Fly,O
2,0,Me,O
3,0,to,O
4,0,Leanin,B-BRAND


In [ ]:
#There are 1 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. Let's print them by frequency (highest to lowest):
tags = {}
for tag, count in zip(frequencies.index, frequencies):
    if tag != "O":
        if tag[2:5] not in tags.keys():
            tags[tag[2:5]] = count
        else:
            tags[tag[2:5]] += count
    continue

print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

[('BRA', 148834), ('', 1541)]


In [ ]:
#create dictionary with numbers, indicies
labels_to_ids = {k: v for v, k in enumerate(data.labels.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.labels.unique())}
labels_to_ids

{'O': 0, 'B-BRAND': 1, 'I-BRAND': 2}

In [ ]:
# fill nan values
data = data.fillna(method='ffill')
data.head()

,sentence_id,words,labels
0,0,,O
1,0,Fly,O
2,0,Me,O
3,0,to,O
4,0,Leanin,B-BRAND


In [ ]:
# let's create a new column called "sentence" which groups the words by sentence
data['sentence'] = data[['sentence_id','words','labels']].groupby(['sentence_id'])['words'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence
data['word_labels'] = data[['sentence_id','words','labels']].groupby(['sentence_id'])['labels'].transform(lambda x: ','.join(x))
data

,sentence_id,words,labels,sentence,word_labels
0,0,,O,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
1,0,Fly,O,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
2,0,Me,O,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
3,0,to,O,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
4,0,Leanin,B-BRAND,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
...,...,...,...,...,...
1139035,98828,Tape,O,"Comix B2250 Office Desk Organizer , 8 Componen...","B-BRAND,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
1139036,98828,Dispenser,O,"Comix B2250 Office Desk Organizer , 8 Componen...","B-BRAND,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
1139037,98828,",",O,"Comix B2250 Office Desk Organizer , 8 Componen...","B-BRAND,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
1139038,98828,Smoky,O,"Comix B2250 Office Desk Organizer , 8 Componen...","B-BRAND,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [ ]:
#select sentence,labels column and drop duplicates
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Fly Me to Leanin ' Tree the Moon - Josephine...,"O,O,O,O,B-BRAND,I-BRAND,I-BRAND,O,O,O,O,O,O,O"
1,Instant Breakfast Complete Nutritional Class...,"O,O,O,O,O,O,O,B-BRAND,O,O,O,O,O,O,O"
2,RI Novelty Slam Dunk Basketball Gumball Dispen...,"B-BRAND,I-BRAND,O,O,O,O,O,O,O"
3,"Spiced Chai Novus Black Tea , 12 - Count Tea...","O,O,O,B-BRAND,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,Mint Julep Cocktail Mixer by Woodford Reserve ...,"O,O,O,O,O,B-BRAND,I-BRAND,O,O,O,O,O,O,O"


# Cross verify the sentence & label

In [ ]:
len(data)

98820

In [ ]:
data.iloc[41].sentence

'  Alphabet Pasta , 7 - Ounce ( Pack La Moderna of 3 )'

In [ ]:
data.sentence = data.sentence.str.strip()

In [ ]:
data.iloc[41].word_labels

'O,O,O,O,O,O,O,O,O,B-BRAND,I-BRAND,O,O,O'

# Preparing the dataset & dataloader

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
#dataset class (which transforms examples of a dataframe to PyTorch tensors
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
#create 2 datasets, one for training and one for testing. Let's use a 80/20 split
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (98820, 2)
TRAIN Dataset: (79056, 2)
TEST Dataset: (19764, 2)


In [ ]:
training_set[0]

{'ids': tensor([  101,  4086,  4589,  1998,  4589,  6546,  5909,  9389,  1011,  1048,
          1005,  8680,  9496,  2368,  1011,  2340,  1012,  1020, 11472,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [ ]:
training_set[0]["ids"]

tensor([  101,  4086,  4589,  1998,  4589,  6546,  5909,  9389,  1011,  1048,
         1005,  8680,  9496,  2368,  1011,  2340,  1012,  1020, 11472,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
sweet       O
orange      O
and         O
orange      O
flower      O
fruit       O
jam         O
-           O
l           B-BRAND
'           B-BRAND
epic        B-BRAND
##uri       B-BRAND
##en        B-BRAND
-           O
11          O
.           O
6           O
oz          O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


In [ ]:
#define pytorch loaders
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Defining the model

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

# Training the model

In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(0.9866, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
#check the shape of batch_size, sequence_length, num_lables
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 3])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.9611827731132507
Training loss per 100 training steps: 0.12757378444075584
Training loss per 100 training steps: 0.09174270281077024
Training loss per 100 training steps: 0.07852612321336205
Training loss per 100 training steps: 0.07137204942634873
Training loss per 100 training steps: 0.06632383660359416
Training loss per 100 training steps: 0.06257253891157727
Training loss per 100 training steps: 0.05968290543749652
Training loss per 100 training steps: 0.05753287664741352
Training loss per 100 training steps: 0.05560714961107113
Training loss per 100 training steps: 0.05374273946313383
Training loss per 100 training steps: 0.052448324363451995
Training loss per 100 training steps: 0.05129660553857995
Training loss per 100 training steps: 0.05028375745074484
Training loss per 100 training steps: 0.04927687790593149
Training loss per 100 training steps: 0.048282429237765004
Training loss per 100 training steps: 0.0473653138530

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.0052770585753023624
Validation loss per 100 evaluation steps: 0.01855647478005987
Validation loss per 100 evaluation steps: 0.01978789855664091
Validation loss per 100 evaluation steps: 0.019093280714834834
Validation loss per 100 evaluation steps: 0.018793671648177735
Validation loss per 100 evaluation steps: 0.018281491464545333
Validation loss per 100 evaluation steps: 0.018131976321552876
Validation loss per 100 evaluation steps: 0.017934394023741047
Validation loss per 100 evaluation steps: 0.01836636372914636
Validation loss per 100 evaluation steps: 0.01809839386242371
Validation loss per 100 evaluation steps: 0.018217984880332938
Validation loss per 100 evaluation steps: 0.01823543408164426
Validation loss per 100 evaluation steps: 0.01867774473252663
Validation loss per 100 evaluation steps: 0.018841724091881118
Validation loss per 100 evaluation steps: 0.0189141025872468
Validation loss per 100 evaluation steps: 0.018900859907815412

# Evaluation Metrics

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

       BRAND       0.79      0.77      0.78     33388

   micro avg       0.79      0.77      0.78     33388
   macro avg       0.79      0.77      0.78     33388
weighted avg       0.79      0.77      0.78     33388



In [ ]:
import os

directory = "/content/drive/MyDrive/Colab Notebooks/Product_Feature_ner/model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


In [ ]:
from transformers import pipeline

pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
pipe("Fly Me to Leanin ' Tree the Moon - Josephine Wall Birthday Card")

[{'entity_group': 'BRAND',
  'score': 0.502654,
  'word': "' tree",
  'start': None,
  'end': None}]